In [67]:
import pandas as pd
import numpy as np

df = pd.read_csv('data-files/movies/movies_metadata.csv')

#Print all the features (or columns) of the DataFrame
df.columns

c:\programdata\miniconda3\envs\hoseo\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [68]:
#Only keep those features that we require 
df = df[['title','genres', 'release_date', 'runtime', 'vote_average', 'vote_count']] # genres는 딕셔너리로 들어가있음

df.head()

,title,genres,release_date,runtime,vote_average,vote_count
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0


In [69]:
print(df['genres'].dtype, type(df['genres'][0]))
df['genres'][0]# animation, comedy, family

object <class 'str'>


"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [70]:
#Convert release_date into pandas datetime format
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce') # 오류가 발생하면 오류 부분 NA값으로 채워라!

#Extract year from the datetime
df['year'] = df['release_date'].apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan) # 년도 - 기준으로 split, nan이면 np.nan으로 처리

In [71]:
#Helper function to convert NaT to 0 and all other years to integers.
def convert_int(x):
    try:
        return int(x)
    except:
        return 0

In [72]:
#Apply convert_int to the year feature
df['year'] = df['year'].apply(convert_int)

In [73]:
#Drop the release_date column
df = df.drop('release_date', axis=1)

#Display the dataframe
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",81.0,7.7,5415.0,1995
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",127.0,6.1,34.0,1995
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",106.0,5.7,173.0,1995


In [74]:
#Print genres of the first movie
df.iloc[0]['genres']

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [75]:
#Import the literal_eval function from ast
from ast import literal_eval

#Define a stringified list and output its type
a = "[1,2,3]" # 문자열 # a[0] -> []
print(type(a))

#Apply literal_eval and output type
b = literal_eval(a) # literal_eval() -> 문자열을 파이선 코드로 처리 # "[1,2,3]" -> [1,2,3] 문자열에서 리스트로. b[0]-> 1
print(type(b))
print(a[0], '-', b[0]) # [ - 1 

<class 'str'>
<class 'list'>
[ - 1


In [76]:
#Convert all NaN into stringified empty lists
df['genres'] = df['genres'].fillna('[]')

#Apply literal_eval to convert stringified empty lists to the list object
df['genres'] = df['genres'].apply(literal_eval) # 리스트로 만들어줌

#Convert list of dictionaries to a list of strings
df['genres'] = df['genres'].apply(lambda x: [i['name'].lower() for i in x] if isinstance(x, list) else []) # name만 뽑아옴

In [77]:
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[animation, comedy, family]",81.0,7.7,5415.0,1995
1,Jumanji,"[adventure, fantasy, family]",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[romance, comedy]",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[comedy, drama, romance]",127.0,6.1,34.0,1995
4,Father of the Bride Part II,[comedy],106.0,5.7,173.0,1995


In [81]:
# df.iloc[0]['genres']
# pd.Series(df.iloc[0]['genres'])
df2 = df.apply(lambda x: pd.Series(x['genres']),axis=1) # 0행 뿐만 아니라 모든 행에 대해 장르 각각을 시리즈로 만들기

c:\programdata\miniconda3\envs\hoseo\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [82]:
df2.head()

,0,1,2,3,4,5,6,7
0,animation,comedy,family,NaN,NaN,NaN,NaN,NaN
1,adventure,fantasy,family,NaN,NaN,NaN,NaN,NaN
2,romance,comedy,NaN,NaN,NaN,NaN,NaN,NaN
3,comedy,drama,romance,NaN,NaN,NaN,NaN,NaN
4,comedy,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
# df2.stack() # 0행 0열~2열, 1행 0~1열 ....을 나타냄. 위의 테이블을 전부 행으로 나타냄.
df2.stack().reset_index(level=1) # default = 앞의 인덱스(stack()의)를 지우고 나타냄 # level을 지정하면 맨 앞(0)을 없앨 건지, 그 뒤를 없앨 건지(level=1) 정하는 것, -1은 맨 뒤의 것 삭제

,level_1,0
0,0,animation
0,1,comedy
0,2,family
1,0,adventure
1,1,fantasy
...,...,...
45461,1,family
45462,0,drama
45463,0,action
45463,1,drama


In [50]:
#Create a new feature by exploding genres
s = df.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True) # stack -> 행을 열로 바꿨다, 열을 행으로 바꿨다 하는 것(여기서는 결과를 행으로 바꿈)# drop -> 변경된 채로 저장 # unstack -> 행을 열로 다시 옮겨주는 것

#Name the new feature as 'genre'
s.name = 'genre'

#Create a new dataframe gen_df which by dropping the old 'genres' feature and adding the new 'genre'.
gen_df = df.drop('genres', axis=1).join(s) # s를 기준으로 join

#Print the head of the new gen_df
gen_df.head() # genre 행을 풀어서 배열하는 작업 (Toy Story의 genre가 딕셔너리 3개였지만 풀어서 배열했음)

c:\programdata\miniconda3\envs\hoseo\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


,title,runtime,vote_average,vote_count,year,genre
0,Toy Story,81.0,7.7,5415.0,1995,animation
0,Toy Story,81.0,7.7,5415.0,1995,comedy
0,Toy Story,81.0,7.7,5415.0,1995,family
1,Jumanji,104.0,6.9,2413.0,1995,adventure
1,Jumanji,104.0,6.9,2413.0,1995,fantasy


In [53]:
def build_chart(gen_df, percentile=0.8):
    #Ask for preferred genres
    print("Input preferred genre")
    genre = input()
    
    #Ask for lower limit of duration
    print("Input shortest duration")
    low_time = int(input())
    
    #Ask for upper limit of duration
    print("Input longest duration")
    high_time = int(input())
    
    #Ask for lower limit of timeline
    print("Input earliest year")
    low_year = int(input())
    
    #Ask for upper limit of timeline
    print("Input latest year")
    high_year = int(input())
    
    #Define a new movies variable to store the preferred movies. Copy the contents of gen_df to movies
    movies = gen_df.copy()
    
    #Filter based on the condition
    movies = movies[(movies['genre'] == genre) & 
                    (movies['runtime'] >= low_time) & 
                    (movies['runtime'] <= high_time) & 
                    (movies['year'] >= low_year) & 
                    (movies['year'] <= high_year)]
    
    #Compute the values of C and m for the filtered movies
    C = movies['vote_average'].mean()
    m = movies['vote_count'].quantile(percentile)
    
    #Only consider movies that have higher than m votes. Save this in a new dataframe q_movies
    q_movies = movies.copy().loc[movies['vote_count'] >= m]
    
    #Calculate score using the IMDB formula
    q_movies['score'] = q_movies.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) 
                                       + (m/(m+x['vote_count']) * C)
                                       ,axis=1)

    #Sort movies in descending order of their scores
    q_movies = q_movies.sort_values('score', ascending=False)
    
    return q_movies

In [55]:
#Generate the chart for top animation movies and display top 5.
build_chart(gen_df).head()

Input preferred genre


 comedy


Input shortest duration


 60


Input longest duration


...

Input earliest year


 2010


Input latest year


 2020


,title,runtime,vote_average,vote_count,year,genre,score
18465,The Intouchables,112.0,8.2,5410.0,2011,comedy,8.137717
22841,The Grand Budapest Hotel,99.0,8.0,4644.0,2014,comedy,7.933503
30315,Inside Out,94.0,7.9,6737.0,2015,comedy,7.855752
24455,Big Hero 6,102.0,7.8,6289.0,2014,comedy,7.754812
38798,Captain Fantastic,119.0,7.9,1569.0,2016,comedy,7.721774


In [56]:
#Convert the cleaned (non-exploded) dataframe df into a CSV file and save it in the data folder
#Set parameter index to False as the index of the DataFrame has no inherent meaning.
df.to_csv('data-files/movies/metadata_clean.csv', index=False)